In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


# Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

# Username and password and CRUD Python module name
username = "aacuser"
password = "AACUser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read())





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Imports Grazioso Logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    
    # Drop down menu that displays animal types
    html.Div(className='row',
             style={'flex-wrap': 'nowrap'},
             children=[dcc.Dropdown(id='dropdown-button', options=[
                 {"label": "Unfiltered List", "value": "All"},
                 {"label": "Water Rescue", "value": "Water"},
                 {"label": "Mountain Rescue", "value": "Mountain"},
                 {"label": "Disaster Rescue", "value": "Disaster"},
             ],
                                    value='All',
                                    clearable=False
                                    )
                       ]),
    # Unique identifier
    html.Center(html.B(html.H3('Bairon Gomez\'s SNHU CS-340 Project 2 (Dashboard)'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Interactive data table to make it user-friendly for the client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,

    ),
    html.Br(),
     html.Hr(),
    # Sets up the dashboard so that the chart and the geolocation chart are side-by-side
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#This callback will filter the table to match the users choice
@app.callback(
    Output('datatable-id', 'data'),
    [Input('dropdown-button', 'value')])
def dropdown_click(radio_value):
    # start case of showing list unfiltered
    if radio_value == "All":
        df = pd.DataFrame(list(shelter.read()))
    # Filters list to show only Water Rescue Type who are intact female between 26 and 156 weeks
    elif radio_value == "Water":
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":
                                             {"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever",
                                                     "Newfoundland"]},"sex_upon_outcome":"Intact Female",
                                             "age_upon_outcome_in_weeks":{"$gte":20},
                                             "age_upon_outcome_in_weeks":{"$lte":156}})))
    # Filters list to show only Mountain or Wilderness Type who are intact Male between 26 and 156 weeks
    elif radio_value == "Mountain":
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":
                                             {"$in":["German Shepherd","Alaskan Malamute",
                                                     "Old English Sheepdog","Siberian Husky","Siberian Husky"]},
                                             "sex_upon_outcome":"Intact Male",
                                             "age_upon_outcome_in_weeks":{"$gte":20},
                                             "age_upon_outcome_in_weeks":{"$lte":156}})))
    # Filters list to show only Disaster or Individual Tracking who are intact Male between 20 and 300 weeks
    elif radio_value == "Disaster":
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog","breed":
                                             {"$in":["Doberman Pinscher","German Shepard",
                                                     "Golden Retriever","Bloodhound","Rottweiler"]},
                                             "sex_upon_outcome":"Intact Male",
                                             "age_upon_outcome_in_weeks":{"$gte":20},
                                             "age_upon_outcome_in_weeks":{"$lte":300}})))
    
    
    return df.to_dict('records')
  
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Convert data to pandas dataframe
        dff = pd.DataFrame.from_dict(viewData)
        # Count by breed occurrence
        new_df = dff.breed.value_counts()
        new_df.name = "breed count"
        new_df = new_df.to_frame()
        new_df.reset_index(inplace=True)
        
        # Returns updated graph data
        return [
        dcc.Graph(
        figure = px.pie(new_df, values="breed count", names="index",title='Percentage of Breeds Available')
        )
        ]
    
# map with market of first row of chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
        dff = pd.DataFrame.from_dict(viewData)
        # Los Angeles, CA is at [34.0522, -118.2437]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], zoom=15, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                    dl.Tooltip(dff.iloc[0, 4]),
                    dl.Popup([
                        # Unique identifier
                        html.H3("Rescue Animal\'s Name: "),
                        html.P(dff.loc[0,'name'])
                    ])
                ])
            ])
        ]

app